This notebook allows for live plotting of data from the PiDAQ

## Import Required Modules

In [ ]:
import time
import pydaqhat as py
from bokeh.io import push_notebook, show, output_notebook
from bokeh.plotting import figure
from bokeh.layouts import gridplot
import time
import ipywidgets as widgets
import os

output_notebook()

## Plot
This cell initialises a blank plot for live plotting.

In [ ]:
from bokeh.events import ButtonClick
from bokeh.models import Button

button = Button()

def callback(event):
    print('Python:Click')

button.on_event(ButtonClick, callback)


In [ ]:
show(button)

In [ ]:
from IPython.display import display
button = widgets.Button(description="Stop Plotting")
output = widgets.Output()

live_file = "/tmp/bokeh_plot.pid"

def on_button_clicked(b):
    with output:
        if os.path.isfile(live_file):
            os.remove(live_file)
        print("Plotting stopped.")

button.on_click(on_button_clicked)

In [ ]:
button.get_state()

In [ ]:
button.on_click?

In [ ]:
p = figure(title="Live Plot", 
           height=300, 
           width=600, 
           background_fill_color='#efefef'
          )
r = p.line(range(500), range(500), color="#8888cc", line_width=1.5, alpha=0.8)

display(button, output)
show(p, notebook_handle=True)
n = 0
while n<10:
    print(n)
    n += 1
    time.sleep(1)

## Start live plot

This cell reads data live from the PiDAQ and plots it to the graph above. Click the stop button in Jupyter to stop plotting. If you want plotting to resume, run the initialisation cell above before this one. 

In [ ]:
 def plot_continous(hat):
    print("Plotting")
    with open(live_file, 'w') as fp:
        pass
    
    sample_count = 0
    x = 0
    while os.path.isfile(live_file):
        x += 1
        new = hat.a_in_scan_read(-1,0).data[-500:]
        
        r.data_source.data = {'x': range(0, len(new)), 'y': new}
        
        sample_count += len(new)

        push_notebook()
        print(x)
        time.sleep(1)
        
    hat.a_in_scan_stop()
    hat.a_in_scan_cleanup()
    print("Scan has stopped")    
    
hat = py.continous_scan_start(
    channels = [0], # Channels to use
    iepe_enable = False, # IEPE enable/disable
    sensitivity = 1000, # Sensitivity in mV/unit
    sample_rate = 20480, # Number of samples per second
    buffer_size = 1000000 # Number of samples to keep in buffer before overwriting  
)

plot_continous(hat)

# Stop continous scan
Run this once finished with this notebook to clean up the PiDAQs internal buffer.

In [ ]:
hat.a_in_scan_stop()
hat.a_in_scan_cleanup()
        
print("Scan has stopped")